In [ ]:
import time

import numpy as np
import pandas as pd
from bstrapping.synthetic_time_series.moving_average import MovingAverage
from bstrapping.weights.moving_average import MovingAverageWeights, triangle_window

In [ ]:
mean = 4  # mean of the time series
alpha = 0.05

# Names of the stochastic processes

# Dependence coefficients of the stochastic processes, i.e. of the moving average processes
dependence_coefficients = [
    np.array([0]),
    np.array([0.5]),
    np.array([0.5 ** i for i in range(1, 3)]),
    np.array([0.5 ** i for i in range(1, 11)]),
    np.array([0.5 ** i for i in range(1, 16)]),
    np.array([0.5 ** i for i in range(1, 21)]),
]

names_dependence_coefficients = [
    "iid",
    "MA(1)",
    "MA(2)",
    "MA(10)",
    "MA(15)",
    "MA(20)",
]

list_name_weights = ['AR',
                     'Multiplier',
                     'MA',
                     ]

In [ ]:
sample_size = 2000
index_dependence = 0
bootstrapped_sample_size = 250
samples = MovingAverage(mean=mean, parameters=dependence_coefficients[index_dependence]).generate_samples(sample_size)

In [ ]:
MovingAverage(mean=mean, parameters=dependence_coefficients[index_dependence]).asymptotic_variance

# Online multiplier bootstrap

In [ ]:
from bstrapping.weights.auto_regressive_weights import AutoRegressiveWeights, generate_recursive_weight
from bstrapping.bootstrap_procedures.weighted_bootstrap import WeightedBootstrap

bootstrap = WeightedBootstrap(samples=samples,
                              weights=AutoRegressiveWeights(samples=samples),
                              number_bootstrap_samples=bootstrapped_sample_size)

In [ ]:
sample_size * bootstrap.bootstrapped_variance

In [ ]:
def new_weights_ar(t, old_weights):
    return np.array([generate_recursive_weight(t, V_i, alpha=2 ** (1 / 2) - 1) for V_i in old_weights])

In [ ]:
def online_update(new_sample, averages_mean, new_weights, sum_old_weights):
    averages_mean = (sum_old_weights / (sum_old_weights + new_weights)) * averages_mean + 1 / (
            sum_old_weights + new_weights) * new_weights * new_sample
    return averages_mean

In [ ]:
duration_ar = []

start_time = time.perf_counter()
old_weights = np.random.normal(loc=1, scale=1, size=bootstrapped_sample_size)
sum_old_weights = old_weights
old_weights = old_weights
averages_mean = np.array([samples[0] * old_weights])
end_time = time.perf_counter()
duration_ar.append(end_time - start_time)
# for validation
asy_var = []
for t, sample in enumerate(samples[1:]):
    start_time = time.perf_counter()
    new_weights = new_weights_ar(t + 1, old_weights)
    averages_mean = online_update(sample, averages_mean, new_weights, sum_old_weights)
    sum_old_weights += new_weights
    # for validation

    asy_var.append((t + 1) * np.var(averages_mean))

    old_weights = new_weights
    end_time = time.perf_counter()
    duration_ar.append(end_time - start_time)

In [ ]:
pd.DataFrame(asy_var)[1000:].plot(title="AR")

# IID

In [ ]:
def new_weights_iid():
    return np.random.normal(loc=1, scale=1, size=bootstrapped_sample_size)


duration_iid = []
# for validation
asy_var = []
averages_mean = np.zeros(bootstrapped_sample_size)
sum_old_weights = 0
for t, sample in enumerate(samples):
    start_time = time.perf_counter()
    new_weights = new_weights_iid()
    averages_mean = online_update(sample, averages_mean, new_weights, sum_old_weights)
    sum_old_weights += new_weights
    # for validation

    asy_var.append((t + 1) * np.var(averages_mean))

    old_weights = new_weights
    end_time = time.perf_counter()
    duration_iid.append(end_time - start_time)

In [ ]:
pd.DataFrame(asy_var)[100:].plot(title="IID")

# MA 

The moving average bootstrap (block multiplier bootstrap) requires updating *all* weights whenever the block length (``ìnt(n**(1/3))``) changes.  In particular, this makes this bootstrap an offline bootstrap. 
However, in order to compare the evaluation times, whenever the block length does not change and 'online' update of the bootstrap is possible. This is implemented in the following.

In [ ]:
def new_weights_ma_same_block_length(gamma_weights, t, block_length):
    return np.sum([
        triangle_window(block_length, j) * gamma_weights[t - j]
        for j in range(-block_length, block_length + 1)])

In [ ]:
duration_ma = []
# for validation

In [ ]:
times = []

asy_var = []
for t, sample in enumerate(samples):
    start_time = time.perf_counter()
    dummy = []
    if int(t ** (1 / 3)) == int((t + 1) ** (1 / 3)): # block length does not change
        for gamma_weight in gamma_weights:
            gamma_weight.append(np.random.gamma(q, 1 / q, ))
        dummy.append(time.perf_counter())  #
        new_weights = new_weights_ma_same_block_length(np.array(gamma_weights).T, t + 1, block_length)
        dummy.append(time.perf_counter())  #
        averages_mean = online_update(sample, averages_mean, new_weights, sum_old_weights)
        sum_old_weights += new_weights
        # for validation

        asy_var.append((t + 1) * np.var(averages_mean))

        old_weights = new_weights
        dummy = np.array(dummy)
        dummy = dummy[1:] - dummy[:-1]
        times.append(dummy)
    else: # block length changes
        print("time index at which block length changed: ", t)
        block_length = int((t + 1) ** (1 / 3))
        q = 2 / (3 * block_length) + 1 / (3 * block_length ** 3)
        new_weights_ma_same_block_length.gamma_weights = []
        gamma_weights = []
        averages_mean = []
        for _ in range(bootstrapped_sample_size):
            weights = MovingAverageWeights(samples=samples[:t + 1])
            weight = weights()
            gamma_weights.append(weights._gamma_weights)
            averages_mean.append(np.average((1 / np.average(weight) * weight * samples[:t + 1])))
        averages_mean = np.array(averages_mean)
        asy_var.append((t + 1) * np.var(averages_mean))

    end_time = time.perf_counter()
    duration_ma.append(end_time - start_time)

In [ ]:
pd.DataFrame(asy_var).plot(title="MA")

# Evaluation

In [ ]:
benchmark = pd.DataFrame([duration_ar, duration_ma, duration_iid],
                         index=["AR", "MA", "IID"],
                         columns=range(1, 1 + sample_size)).T

benchmark.index_name = "Sample size"

In [ ]:
benchmark

In [ ]:
benchmark["IID"].plot(title="Online IID bootstrap", logy=True)

In [ ]:
benchmark["MA"].plot(title="MA bootstrap", ylim=(0, 0.1))

In [ ]:
benchmark["AR"].plot(title="Online AR bootstrap")

In [ ]:
print(benchmark.mean())
benchmark.mean().plot()

In [ ]:
#benchmark.to_csv(f"./data/benchmark_time.csv")
#benchmark.to_pickle(f"./data/benchmark_time.pkl")